# DSA Annotation

Digital Slide Archive (DSA) is an open-source web application where users can annotate regional and point annotations on the high power slide viewer. Luna Pathology CLIs pull the different annotation types from DSA, and save the annotations in GeoJSON format along with metadata. In this notebook, we will review:

- Project setup on DSA
- Create annotations on DSA
- Run regional annotation ETL
- Run point annotation ETL

DSA provides an excellent [video tutorial](https://www.youtube.com/watch?v=HTvLMyKYyGs&ab_channel=DigitalSlideArchive%2FHistomicsTK) that covers platform features. For the first two points on DSA, the information below is an abridged version of the tutorial for your reference.

## Project setup on DSA

Digital Slide Archive (DSA) is a platform that provides the ability to store, manage, visualize and annotate large imaging data sets. The DSA consists of an interface to visualize slides and manage annotations (HistomicsUI), and a web-server that provides a rich API and data management tools (using Girder). This system can:

- Organize images from a variety of assetstores, such as local files systems and S3.
- Provide user access controls.
- Image annotation and review.

HistomicsUI is a web-based application for examining, annotating, and processing histology images to extract both low and high level features (e.g. cellular structure, feature types).
Concepts

- **Collections** correspond to a project. Collections are at the top level objects in the data organization hierarchy.
- **Folders** help organize slides under a project. e.g. hne_slides
- **Items** correspond to a slide. An item can have metadata, annotations and files associated with it.
- **Annotation** is a single rectangle, point, or polygon
- **Annotation Document** is a set of annotations, created by the pathologist.
- **Annotation Style** is a predefined set of labels (morphology like tumor, stroma, necrosis etc) and colors.

Create a collection for your project.
Your images can be organized in a folder.
In this example, we have a `pathology-tutorial` collection with `slides` folder where we organized the images.

<img src="../img/dsa-collection-screenshot.png" alt="DSA Collection" width="600px" />


## Create annotations on DSA

Please see this [video tutorial](https://youtu.be/HTvLMyKYyGs?t=369) for creating and viewing annotations. The information below is an abridged version of the tutorial for your reference. 

**1. To navigate to HistomicsUI, go to the Actions → Open in HistomicsUI on the upper right side. HistomicsUI will open a new tab in your browser.**

<img src="../img/dsa-histomicsui-screenshot.png" alt="DSA Collection" width="400px" />

**2. Create an annotation document**
- Click on the + New button on the Annotation panel. This will bring up a Create annotation modal.
- Name you annotation document **regional** or **point**. These are the two types of annotations we support. The annotation document name will be used in the ETL, it is important to standardize your document names so the ETL can download all documents for the annotation type.
- Optionally add a description, then click save.
    
<img src="../img/dsa-document-screenshot.png" alt="DSA Document" width="400px" />

**3. Create annotations**

- Select a label (e.g. regional_tumor)
- Click on **Point** or **Polygon**. When an annotation shape is highlighted, then your cursor on the slide area will look like a +
- For Point annotation, zoom to an appropriate magnification and click on the cell. The annotation will appear as a circle.
- For Polygon annotation, click and drag your mouse. As you drag the area will be highlighted. Try to meet the starting point, or double click to close the polygon.

<img src="../img/dsa-annotation-screenshot.png" alt="DSA Annotation" width="200px" />


**Note**: Using standardized annotation styles is recommended. A uniform annotation style json can be created and shared among the pathologists making annotations.

## Run regional annotation ETL


In [ ]:
import os
HOME = os.environ['HOME']

Once you have created annotations on DSA, we can run the annotation ETL CLI! This ETL will download the annotations, convert them to GeoJSON format, and create a parquet table to make the annotations and metadata queryable.

For details of the data and app configuration, please refer to the example configurations.

**Note**: details of your DSA instance should be updated to reflect your DSA setup. If you are using the luna tutorial docker, replace the `localhost` with the IP you get from running:

```docker inspect -f '{{range.NetworkSettings.Networks}}{{.IPAddress}}{{end}}' luna_tutorial_girder_1```

First, let's look at the CLI arguments, by running `--help`

In [ ]:
!dsa_annotation --help

In [ ]:
# ingest annotations
!dsa_annotation http://localhost:8080/api/v1 \
--output_dir ../dsa_annotations \
--collection_name 'TCGA collection' \
--annotation_name ov_regional \
--num_cores 1 \
--username admin --password password

In [ ]:
# metadata, geojson, parquet table output
!ls -lh ../dsa_annotations/

Annotations are saved in a parquet format, where 1 row represents an annotation element.

We collect metadata about the annotation such as created timestamp and user.
Note that different annotation types (point, regional) can be ingested using the same CLI

In [ ]:
# check annotation metadata table
import pyarrow.parquet as pq

annotation_table = pq.read_table(r'../dsa_annotations/slide_annotation_dataset_TCGA collection_ov_regional.parquet').to_pandas()
print(annotation_table.columns)
annotation_table